In [5]:
import secrets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import radcad as rc

from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend
from random import randint, uniform

sns.set(style="whitegrid")

In [8]:
%%capture
constants = {
    # 1559 fee mechanism params 
    "BLOCK_RESOURCE_LIMITS": 30e6,
    "BLOCK_RESOURCE_TARGETS": 15e6,
    "BASEFEE_MAX_CHANGE_DENOMINATOR": 8,
    
    # 4844 fee mechanism params
    "MAX_DATA_GAS_PER_BLOCK": 2**19,
    "TARGET_DATA_GAS_PER_BLOCK": 2**18,
    "DATA_GAS_PER_BLOB": 2**17,
    "MIN_DATA_GASPRICE": 1,
    "DATA_GASPRICE_UPDATE_FRACTION": 2225652,
}

# helper function approximating exponential with taylor
def fake_exponential(factor: int, numerator: int, denominator: int) -> int:
    i = 1
    output = 0
    numerator_accum = factor * denominator
    while numerator_accum > 0:
        output += numerator_accum
        numerator_accum = (numerator_accum * numerator) // (denominator * i)
        i += 1
    return output // denominator

def calc_price_data(excess_data_gas) -> int:
    return fake_exponential(
        constants["MIN_DATA_GASPRICE"],
        excess_data_gas,
        constants["DATA_GASPRICE_UPDATE_FRACTION"]
    )

class Transaction:
    def __init__(self, max_priority_fee_per_gas, gas_used, max_fee_per_gas):
        self.tx_hash = secrets.token_bytes(6).hex()
        self.max_fee_per_gas = max_fee_per_gas
        self.max_priority_fee_per_gas = max_priority_fee_per_gas
        self.gas_used = gas_used

    def is_valid(self, price, price_data):
        return self.max_fee_per_gas >= price
    
    def get_premium(self, price):
        priority_fee = min(self.max_fee_per_gas - price, self.max_priority_fee_per_gas)
        return self.gas_used * priority_fee

class BlobTransaction(Transaction):
    def __init__(self, max_priority_fee_per_gas, gas_used, max_fee_per_gas, max_fee_per_data_gas, blob_hashes):
        super().__init__(max_priority_fee_per_gas, gas_used, max_fee_per_gas)
        self.max_fee_per_data_gas = max_fee_per_data_gas
        self.blob_hashes = blob_hashes
    
    def is_valid(self, price, price_data):
        return self.max_fee_per_gas >= price and self.max_fee_per_data_gas >= price_data

class Block1():
    def __init__(self, txs):
        self.txs = txs
        
# 1559 fee mechanism
def update_price(params, step, h, s, _input):
    block = _input["block"]
    price = s["price"]
    
    target = constants["BLOCK_RESOURCE_TARGETS"]
    utilized = sum([tx.gas_used for tx in block.txs]) 
    new_price = price * (1 + (utilized - target) / (target * constants["BASEFEE_MAX_CHANGE_DENOMINATOR"]))
    
    return ("price", new_price)

# 4844 fee mechanism
def update_excess_data_gas(params, step, h, s, _input):
    block = _input["block"]
    excess_data_gas = s["excess_data_gas"]
    
    target = constants["TARGET_DATA_GAS_PER_BLOCK"]
    utilized = constants["DATA_GAS_PER_BLOB"] * sum([tx.blob_hashes if isinstance(tx, BlobTransaction) else 0 for tx in block.txs]) 
    new_excess_data_gas = max(excess_data_gas + (utilized - target), 0)

    return ("excess_data_gas", new_excess_data_gas)

def build_block(params, step, h, s):
    demand = s["demand"]
    price = s["price"]
    excess_data_gas = s["excess_data_gas"]
    price_data = calc_price_data(excess_data_gas)
    
    # select valid transactions and sort them by total gas premium
    # for BlobTransaction we check validity against price_data too
    sorted_valid_demand = sorted(
        [tx for tx_hash, tx in demand.items() if tx.is_valid(price, price_data)],
        key = lambda tx: -tx.get_premium(price)
    )
    
    gas_limit = constants["BLOCK_RESOURCE_LIMITS"]
    data_limit = constants["MAX_DATA_GAS_PER_BLOCK"]
    included_transactions = []
    utilized = 0
    utilized_data = 0
    for tx in sorted_valid_demand:
        if utilized<=gas_limit:
            if isinstance(tx, BlobTransaction) and utilized_data<=data_limit:
                included_transactions.append(tx)
                utilized += tx.gas_used
                utilized_data += tx.blob_hashes * constants["DATA_GAS_PER_BLOB"]
            elif isinstance(tx, BlobTransaction) and utilized_data>data_limit:
                continue
            else:
                included_transactions.append(tx)
                utilized += tx.gas_used
        else:
            break

    return ({"block": Block1(txs=included_transactions)})


def record_latest_block(params, step, h, s, _input):
    block = _input["block"]
    return ("latest_block", block)

def generate_demand(params, step, h, s, _input):

    demand = {}  
    for i in range(1000):
        tx = Transaction(
            gas_used = 50000,
            max_fee_per_gas = 100*uniform(0.75,1.25),
            max_priority_fee_per_gas = uniform(1, 10), # ~1/10 of max fee to represent builder "cost"
        )
        demand[tx.tx_hash] = tx

    for i in range(10):
        tx = BlobTransaction(
            gas_used = 50000,
            max_fee_per_gas = 150*uniform(0.75,1.25),
            max_priority_fee_per_gas = uniform(1, 10),
            max_fee_per_data_gas = 50*uniform(0.75,1.25),
            blob_hashes = 2
        )
        demand[tx.tx_hash] = tx

    return ("demand", demand)


psub = [{
    "policies": {},
    "variables": {
        "demand": generate_demand
    }
}, {
    "policies": {
        "action": build_block
    },
    "variables": {
        "excess_data_gas": update_excess_data_gas,
        "price": update_price,
        "latest_block": record_latest_block,
    }
}]

initial_conditions = {
    "price": 1,
    "excess_data_gas": 0,
    "demand": {},
    "latest_block": Block1(txs=[])
}

simulation_parameters = {
    'T': 100,
    'N': 1,
    'M': {}
}

model = Model(
    initial_state=initial_conditions,
    state_update_blocks=psub,
    params=simulation_parameters
)

simulation = Simulation(
    model=model,
    timesteps=simulation_parameters['T'],
    runs=simulation_parameters['N']
)

experiment = Experiment(simulations=[simulation])
experiment.engine=Engine(backend=Backend.PATHOS, drop_substeps=True)
res = experiment.run()
df = pd.DataFrame()

PicklingError: Can't pickle <class '__main__.Block1'>: attribute lookup Block1 on __main__ failed